In [1]:
import os
import csv
import cv2
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import KFold
import random

import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.models import resnet50

from src.model import DaconModel

from utils.imageprocess import image_transformer, image_processor
from utils.EarlyStopping import EarlyStopping
from utils.dataloader import CustomDataLoader
from utils.radams import RAdam

from tqdm import tqdm
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.INFO)

In [2]:
# Early Stopper
from utils.EarlyStopping import EarlyStopping

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Current Device: {device}")

Current Device: cuda:0


In [26]:
transform_dict = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((40, 40)),
        transforms.RandomChoice([transforms.RandomRotation(30, expand=False), 
                                 transforms.RandomPerspective()]),
        transforms.Pad(108, fill=0, padding_mode='constant'),
        transforms.Normalize([0.5], [0.5]),
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((40, 40)),
        transforms.Pad(108, fill=0, padding_mode='constant'),
        transforms.Normalize([0.5], [0.5]),
    ])
}

trainset = torchvision.datasets.EMNIST(root='./dataset/emnist_dataset',
                                       split='letters',
                                       train=True,
                                       download=True,
                                       transform=transform_dict['train'])

testset = torchvision.datasets.EMNIST(root='./dataset/emnist_dataset',
                                      split='letters',
                                      train=False,
                                      download=True,
                                      transform=transform_dict['test'])

In [27]:
# train_ratio = 0.8
# train_len = int(len(trainset) * train_ratio)
# val_len = len(trainset) - train_len
# logger.info(f'Train Length: {train_len}, Validation Length: {val_len}')

In [28]:
# train_set, val_set = torch.utils.data.random_split(trainset, lengths=[train_len, val_len])

In [35]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
#val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

In [36]:
def weights_init(m):
    classname = m.__class__.__name__
    
    if classname.find('Conv') != -1:
        torch.nn.init.kaiming_normal_(m.weight)
        
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias)
        
    elif classname.find('Linear') != -1:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


def train_model(model=None, epochs=50, lr=50, *loaders):
    
    #model.apply(weights_init)
    
    # Dataloaders (train/val)
    train_loader = loaders[0]
    val_loader = loaders[1]
    
    # Early Stopper
    early_stopping = EarlyStopping(patience=3, verbose=False, path='./pretrained_resnet.pth')
    
    loss_function = nn.CrossEntropyLoss()
    #optimizer = optim.Adam(model.parameters(), lr=lr)
    optimizer = RAdam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), weight_decay=1e-4)

    decayRate = 0.998
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)
    
    train_tot_num = train_loader.dataset.__len__()
    val_tot_num = val_loader.dataset.__len__()
    
    train_loss_sum, train_acc_sum = 0.0, 0.0
    val_loss_sum, val_acc_sum = 0.0, 0.0
    
    logger.info(f'Training begins: epochs={epochs}')
    for epoch in range(epochs):
        
        torch.cuda.empty_cache()
        model.train()
        
        train_tmp_corrects_sum, train_tmp_len = 0, 0
        for idx, (train_X, train_Y) in enumerate(train_loader):    
            
            train_tmp_len += len(train_Y)
            
            train_X = train_X.to(device)
            train_Y = train_Y.to(device)
            
            optimizer.zero_grad()
            
            train_pred = model(train_X)
            train_loss = loss_function(train_pred, train_Y-1)
            train_loss.backward()
            optimizer.step()
            
            # Accuracy
            pred_labels = torch.argmax(train_pred, axis=1)
            train_acc = (pred_labels==(train_Y-1)).sum()
            
            train_acc_sum += train_acc.item()
            train_loss_sum += train_loss.item()
            
            train_tmp_corrects_sum += train_acc.item()
            
            # Check between batches
            if (idx+1) % 100 == 0:
                print(f"----- {str((idx+1)).zfill(4)}th batch | Train Acc: {train_tmp_corrects_sum/train_tmp_len*100:.4f}%")
                
                # initialization
                train_tmp_corrects_sum, train_tmp_len = 0, 0
            
        with torch.no_grad():
            
            for idx, (val_X, val_Y) in enumerate(val_loader):
                
                val_X, val_Y = val_X.to(device), val_Y.to(device)
                val_pred = model(val_X)
                val_loss = loss_function(val_pred, val_Y-1)
                
                val_pred_labels = torch.argmax(val_pred, axis=1)
                val_acc = (val_pred_labels==(val_Y-1)).sum()
                
                val_acc_sum += val_acc.item()
                val_loss_sum += val_loss.item()
        
        train_accuracy = train_acc_sum / train_tot_num * 100
        val_accuracy = val_acc_sum / val_tot_num * 100
        
        print(f"Epoch: {epoch+1} | Training Loss: {train_loss_sum/len(train_loader):.4f} | Val Loss: {val_loss_sum/len(val_loader):.4f} | " \
              f"Train Acc: {train_accuracy:.2f}% | Val Acc: {val_accuracy:.2f}%")
        
        train_loss_sum, train_acc_sum = 0.0, 0.0
        val_loss_sum, val_acc_sum = 0.0, 0.0
        
        early_stopping(-val_accuracy, model)

        if early_stopping.early_stop:
            print("Early stopping condition met --- TRAINING STOPPED")
            break
        
        lr_scheduler.step()
        
    return model

In [37]:
learning_rate = 0.0037
epochs = 50

resnet_model = resnet50().to(device)
resnet_model.fc = nn.Sequential(
    nn.Linear(2048, 256),
    nn.Linear(256, 26),
)

In [38]:
class PretrainedModel(nn.Module):
    def __init__(self):
        super(PretrainedModel, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(1, 3, 1, stride=1),
            nn.ReLU(),
            
            resnet_model,
        )

    def forward(self, x):
        out = self.block(x)
        return out

# 모델 선언
model = PretrainedModel().to(device)

In [64]:
model.block[2].fc[1]

Linear(in_features=256, out_features=26, bias=True)

In [39]:
model_fit = train_model(model, epochs, learning_rate, train_loader, test_loader)

2021-02-04 17:54:20,789 : Training begins: epochs=50
/home/sks/COMPETITION/DACON/computer_vision2/utils/radams.py:45: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


----- 0100th batch | Train Acc: 14.4375%
----- 0200th batch | Train Acc: 33.3438%
----- 0300th batch | Train Acc: 40.6250%
----- 0400th batch | Train Acc: 46.2812%
----- 0500th batch | Train Acc: 53.0000%
----- 0600th batch | Train Acc: 59.8125%
----- 0700th batch | Train Acc: 62.1562%
----- 0800th batch | Train Acc: 64.9375%
----- 0900th batch | Train Acc: 68.2500%
----- 1000th batch | Train Acc: 67.9688%
----- 1100th batch | Train Acc: 72.9062%
----- 1200th batch | Train Acc: 73.2812%
----- 1300th batch | Train Acc: 76.2500%
----- 1400th batch | Train Acc: 75.7500%
----- 1500th batch | Train Acc: 75.6250%
----- 1600th batch | Train Acc: 77.4688%
----- 1700th batch | Train Acc: 79.0000%
----- 1800th batch | Train Acc: 79.3438%
----- 1900th batch | Train Acc: 81.8750%
----- 2000th batch | Train Acc: 80.3438%
----- 2100th batch | Train Acc: 79.2812%
----- 2200th batch | Train Acc: 81.6875%
----- 2300th batch | Train Acc: 81.9375%
----- 2400th batch | Train Acc: 84.1250%
----- 2500th bat

In [44]:
model_fit

PretrainedModel(
  (block): Sequential(
    (0): Conv2d(1, 3, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum

In [ ]:
model_resnet = resnet50(pretrained=True)

In [ ]:
model_resnet.fc = nn.Linear()